<a href="https://colab.research.google.com/github/DiegoAnas/SNN-NLP/blob/master/Sentiment%20CNN%20multibranch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

In [22]:
import tensorflow
print(tensorflow.__version__)
import tensorflow.keras as keras

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import imdb
from tensorflow.keras.layers import Dense, Dropout, Activation, concatenate, Input, \
 Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb

import numpy as np
from sklearn.model_selection import StratifiedKFold

2.1.0-rc1


In [23]:
# Parameters:
# Word Embedding
max_features = 10000
maxlen = 400
embedding_dims = 250

# Convolution
kernel_sizes = [3, 4, 5]  # or filter length
filters = 250 
pool_size = 4 

# Training
batch_size = 30
epochs = 2

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
x_data = np.concatenate((x_train, x_test))
y_data = np.concatenate((y_train, y_test))
skf = StratifiedKFold(n_splits=5, random_state=42)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)


In [0]:
# Network Architecture
def create_conv_mb_model(activation:str, branch_filters:[int], batch_normalize:bool=False,
                         dropout:bool=False, dropout_rate:float=0.1, embedding_dropout:bool=False, 
                         embedding_dropout_rate:float=0.1):
  # Multi-branch CNN
  var_input = Input(shape=(400,))
  emb_layer = Embedding(max_features,
                      embedding_dims,
                      input_length=maxlen)(var_input)
  if embedding_dropout:
    emb_layer = SpatialDropout1D(rate=embedding_dropout_rate)(emb_layer)
  conv_branches = []
  for filter_length in branch_filters:
    conv_layer_1 = Conv1D(filters, kernel_size=filter_length, padding='valid', 
                          activation=activation, strides=1)(emb_layer)
    if batch_normalize:
          conv_layer_1 = BatchNormalization()(conv_layer_1)
    maxpooling = GlobalMaxPooling1D()(conv_layer_1)
    conv_branches.append(maxpooling)

  merge_layer = concatenate(conv_branches)
  if dropout:
      merge_layer = Dropout(rate=dropout_rate, seed=42)(merge_layer)
  dense_layer = Dense(1, activation='sigmoid')(merge_layer)
  model = Model(inputs=var_input, outputs=dense_layer)
  model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
  return model

In [14]:
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='relu', branch_filters=kernel_sizes)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 68s 2ms/sample - loss: 0.3006 - accuracy: 0.8676 - val_loss: 0.2285 - val_accuracy: 0.9068
Epoch 2/2
40000/40000 [==============================] - 62s 2ms/sample - loss: 0.1334 - accuracy: 0.9512 - val_loss: 0.2330 - val_accuracy: 0.9117
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.3003 - accuracy: 0.8679 - val_loss: 0.2455 - val_accuracy: 0.9007
Epoch 2/2
40000/40000 [==============================] - 58s 1ms/sample - loss: 0.1327 - accuracy: 0.9516 - val_loss: 0.2551 - val_accuracy: 0.9086
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 59s 1ms/sample - loss: 0.2995 - accuracy: 0.8696 - val_loss: 0.2608 - val_accuracy: 0.8928
Epoch 2/2
40000/40000 [==============================] - 58s 1ms/sample - loss: 0.1358 - accuracy: 0.9496 - va

In [19]:
#Test w dropout
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='relu', branch_filters=kernel_sizes,
                                dropout=True)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 62s 2ms/sample - loss: 0.3058 - accuracy: 0.8669 - val_loss: 0.2493 - val_accuracy: 0.9000
Epoch 2/2
40000/40000 [==============================] - 60s 2ms/sample - loss: 0.1466 - accuracy: 0.9447 - val_loss: 0.2517 - val_accuracy: 0.9068
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 61s 2ms/sample - loss: 0.3047 - accuracy: 0.8657 - val_loss: 0.2304 - val_accuracy: 0.9071
Epoch 2/2
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.1424 - accuracy: 0.9480 - val_loss: 0.2423 - val_accuracy: 0.9070
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 61s 2ms/sample - loss: 0.3093 - accuracy: 0.8622 - val_loss: 0.2261 - val_accuracy: 0.9102
Epoch 2/2
40000/40000 [==============================] - 60s 2ms/sample - loss: 0.1490 - accuracy: 0.9451 - va

0.1 Dropout doesn't seem to have any impact on the accuracy results.


In [20]:
#Test w SELU
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='selu', branch_filters=kernel_sizes)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 60s 1ms/sample - loss: 0.3003 - accuracy: 0.8696 - val_loss: 0.2238 - val_accuracy: 0.9081
Epoch 2/2
40000/40000 [==============================] - 58s 1ms/sample - loss: 0.1336 - accuracy: 0.9516 - val_loss: 0.2324 - val_accuracy: 0.9112
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 56s 1ms/sample - loss: 0.2994 - accuracy: 0.8670 - val_loss: 0.2323 - val_accuracy: 0.9050
Epoch 2/2
40000/40000 [==============================] - 55s 1ms/sample - loss: 0.1374 - accuracy: 0.9498 - val_loss: 0.2412 - val_accuracy: 0.9074
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.3037 - accuracy: 0.8658 - val_loss: 0.2390 - val_accuracy: 0.9063
Epoch 2/2
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.1345 - accuracy: 0.9507 - va

As expected, SELU activation in such a shallow network does not have a notable impact on accuracy.



In [25]:
#Test w 0.2 dropout
for train, test in skf.split(x_data, y_data):
  target = create_conv_mb_model(activation='relu', branch_filters=kernel_sizes,
                                dropout=True, dropout_rate=0.2)
  target.fit(x_data[train], y_data[train],
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(x_data[test], y_data[test]))

Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 59s 1ms/sample - loss: 0.3141 - accuracy: 0.8608 - val_loss: 0.3520 - val_accuracy: 0.8543
Epoch 2/2
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.1610 - accuracy: 0.9374 - val_loss: 0.2259 - val_accuracy: 0.9154
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 58s 1ms/sample - loss: 0.3124 - accuracy: 0.8613 - val_loss: 0.2394 - val_accuracy: 0.9060
Epoch 2/2
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.1584 - accuracy: 0.9416 - val_loss: 0.2439 - val_accuracy: 0.9054
Train on 40000 samples, validate on 10000 samples
Epoch 1/2
40000/40000 [==============================] - 57s 1ms/sample - loss: 0.3120 - accuracy: 0.8613 - val_loss: 0.2308 - val_accuracy: 0.9080
Epoch 2/2
40000/40000 [==============================] - 56s 1ms/sample - loss: 0.1613 - accuracy: 0.9381 - va